In [2]:
!pip install ISLP

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 42.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 6.4 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=65e00fd18155b5758ffa15e6a35f16b95c9ad1ba10f9ad1b4ac21032f1cfe529
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma
  Attempting un

# 9. In this exercise, we will predict the number of applications received using the other variables in the College data set.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error

(a) Split the data set into a training set and a test set.

In [3]:
from ISLP import load_data
college = load_data("College")
X = college.drop("Apps", axis=1)
y = college["Apps"]
X = pd.get_dummies(X, drop_first=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

(b) Fit a linear model using least squares on the training set, and report the test error obtained.

In [4]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_test)
mse_linear = mean_squared_error(y_test, y_pred_linear)
print("Linear Model Test MSE:", mse_linear)

Linear Model Test MSE: 1659682.1719133756


(c) Fit a ridge regression model on the training set, with λ chosen by cross-validation. Report the test error obtained.

In [5]:
ridge_model = RidgeCV(alphas=np.logspace(-6, 6, 13), cv=5)
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
print("Ridge Model Test MSE:", mse_ridge)
print("Best Lambda for Ridge:", ridge_model.alpha_)

Ridge Model Test MSE: 1642456.265965162
Best Lambda for Ridge: 10.0


(d) Fit a lasso model on the training set, with λ chosen by crossvalidation. Report the test error obtained, along with the number of non-zero coefficient estimates.

In [6]:
lasso_model = LassoCV(cv=5)
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
print("Lasso Model Test MSE:", mse_lasso)
print("Best Lambda for Lasso:", lasso_model.alpha_)
print("Number of Non-zero Coefficients (Lasso):", np.sum(lasso_model.coef_ != 0))

Lasso Model Test MSE: 1914912.3699589907
Best Lambda for Lasso: 56401.126011064014
Number of Non-zero Coefficients (Lasso): 5


(e) Fit a PCR model on the training set, with M chosen by crossvalidation. Report the test error obtained, along with the value of M selected by cross-validation.

In [7]:
# (e) 主成分回歸 (PCR) 模型，使用交叉驗證選擇最佳成分數 M
pca = PCA()
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

In [8]:
# 使用不同的主成分數進行交叉驗證選擇最佳 M
mse_pcr = []
for m in range(1, X_train.shape[1] + 1):
    linear_model.fit(X_train_pca[:, :m], y_train)
    y_pred = linear_model.predict(X_test_pca[:, :m])
    mse_pcr.append(mean_squared_error(y_test, y_pred))

best_m_pcr = np.argmin(mse_pcr) + 1
print("PCR Model Test MSE:", mse_pcr[best_m_pcr - 1])
print("Best Number of Components for PCR:", best_m_pcr)

PCR Model Test MSE: 1598862.8989769155
Best Number of Components for PCR: 16


(f) Fit a PLS model on the training set, with M chosen by crossvalidation. Report the test error obtained, along with the value of M selected by cross-validation.

In [9]:
# (f) 偏最小二乘回歸 (PLS) 模型，使用交叉驗證選擇最佳成分數 M
pls_mse = []
for m in range(1, X_train.shape[1] + 1):
    pls_model = PLSRegression(n_components=m)
    pls_model.fit(X_train, y_train)
    y_pred_pls = pls_model.predict(X_test)
    mse_pls = mean_squared_error(y_test, y_pred_pls)
    pls_mse.append(mse_pls)

best_m_pls = np.argmin(pls_mse) + 1
print("PLS Model Test MSE:", pls_mse[best_m_pls - 1])
print("Best Number of Components for PLS:", best_m_pls)

PLS Model Test MSE: 1659682.1719133682
Best Number of Components for PLS: 17


(g) Comment on the results obtained. How accurately can we predict the number of college applications received? Is there much difference among the test errors resulting from these five approaches?

In [10]:
print("\nModel Comparison:")
print("Linear Model Test MSE:", mse_linear)
print("Ridge Model Test MSE:", mse_ridge, "with Best Lambda:", ridge_model.alpha_)
print("Lasso Model Test MSE:", mse_lasso, "with Best Lambda:", lasso_model.alpha_)
print("PCR Model Test MSE:", mse_pcr[best_m_pcr - 1], "with Best M:", best_m_pcr)
print("PLS Model Test MSE:", pls_mse[best_m_pls - 1], "with Best M:", best_m_pls)


Model Comparison:
Linear Model Test MSE: 1659682.1719133756
Ridge Model Test MSE: 1642456.265965162 with Best Lambda: 10.0
Lasso Model Test MSE: 1914912.3699589907 with Best Lambda: 56401.126011064014
PCR Model Test MSE: 1598862.8989769155 with Best M: 16
PLS Model Test MSE: 1659682.1719133682 with Best M: 17


The ridge model achieves the lowest test error, with the lasso model following closely. Both models outperform PCR and PLS. However, the test error differences across all five methods are minimal.